In [2]:
!pip install pyvi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split,GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, make_scorer

from pyvi import ViTokenizer

import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


MÁY HỌC

In [5]:
df = pd.read_csv("/content/drive/MyDrive/NLP_Đồ án/film_cleaned_ver2.csv")
df["tokenized_text"] = df["cleaned_text"].apply(
    lambda x: ViTokenizer.tokenize(x)
)
dfML = df[df["label"].isin(
    ["negative", "mixed/neutral", "positive"]
)].copy()
dfML['tokenized_text'].head()

0    ta nói nó hài bản_thân tôi con_gái nhưng rất t...
1    hên coi ở cgv phim gì nhân_vật nói chả nghe cá...
2    phim hài cười banh rạp = nửa tiếng đầu mình bi...
3    nếu bỏ tiền mua vé xem phim việt tui tốn chưa ...
4    phim hay ý_nghĩa cười rất nhiều có nhìu hành_đ...
Name: tokenized_text, dtype: object

In [7]:
df.shape

(5409, 10)

In [13]:
X = dfML["tokenized_text"]
y = dfML["label"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [14]:
cv = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=42
)

f1_macro = make_scorer(f1_score, average="macro")

In [ ]:
pipeline = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", LogisticRegression(
        max_iter=1000,
        class_weight="balanced"
    ))
])

In [ ]:
param_grid = {
    "tfidf__ngram_range": [(1,1), (1,2)],
    "tfidf__min_df": [3, 5],
    "tfidf__max_features": [10000, 20000, 30000]
}

In [ ]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring="f1_macro",
    cv=cv,
    n_jobs=-1,
    verbose=1
)

grid.fit(X_train, y_train)

In [ ]:
df_tfidf_cv = pd.DataFrame(grid.cv_results_)
df_tfidf_cv["ngram"] = df_tfidf_cv["params"].apply(lambda x: x["tfidf__ngram_range"])
df_tfidf_cv["min_df"] = df_tfidf_cv["params"].apply(lambda x: x["tfidf__min_df"])
df_tfidf_cv["max_features"] = df_tfidf_cv["params"].apply(lambda x: x["tfidf__max_features"])
df_tfidf_cv["mean_f1"] = df_tfidf_cv["mean_test_score"]


In [ ]:
plt.figure(figsize=(7,5))

sns.barplot(
    data=df_tfidf_cv,
    x="min_df",
    y="mean_f1",
    hue="ngram"
)

plt.xlabel("min_df")
plt.ylabel("Mean F1-macro (CV)")
plt.title("Effect of min_df on TF-IDF performance")
plt.ylim(0, 1)
plt.tight_layout()
plt.show()


In [ ]:
df_tfidf_cv[
    ["ngram", "min_df", "max_features", "mean_f1", "rank_test_score"]
].sort_values("mean_f1", ascending=False)


In [ ]:
print("TF-IDF TUNING RESULT")
print("Best params:")
print(grid.best_params_)
print("\n Best score:")
print(grid.best_score_)

In [10]:
best_tfidf_params = {
    "ngram_range": (1, 2),
    "min_df": 3,
    "max_features": 10000
}

### Logistics Regression Tuning

In [15]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    return {
        "Accuracy": accuracy_score(y_test, y_pred),
        "F1_macro": f1_score(y_test, y_pred, average="macro")
    }

In [16]:
pipe_lr_base = Pipeline([
    ("tfidf", TfidfVectorizer(**best_tfidf_params)),
    ("clf", LogisticRegression(
        max_iter=1000,
        class_weight="balanced"
    ))
])

pipe_lr_base.fit(X_train, y_train)

lr_base_result = evaluate_model(pipe_lr_base, X_test, y_test)
lr_base_result

{'Accuracy': 0.7923228346456693, 'F1_macro': 0.7796054934502262}

In [17]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold

param_grid_lr = {
    "clf__C": [0.01, 0.1, 1, 10],
    "clf__penalty": ["l2"],
    "clf__solver": ["liblinear", "lbfgs"]
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_lr = GridSearchCV(
    pipe_lr_base,
    param_grid_lr,
    scoring="f1_macro",
    cv=cv,
    n_jobs=-1
)

grid_lr.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(max_features=10000,
                                                        min_df=3,
                                                        ngram_range=(1, 2))),
                                       ('clf',
                                        LogisticRegression(class_weight='balanced',
                                                           max_iter=1000))]),
             n_jobs=-1,
             param_grid={'clf__C': [0.01, 0.1, 1, 10], 'clf__penalty': ['l2'],
                         'clf__solver': ['liblinear', 'lbfgs']},
             scoring='f1_macro')

In [18]:
print(grid_lr.best_params_)
print(grid_lr.best_score_)

{'clf__C': 1, 'clf__penalty': 'l2', 'clf__solver': 'lbfgs'}
0.8060825860546575


In [ ]:
df_lr_cv = pd.DataFrame(grid_lr.cv_results_)

df_lr_cv = df_lr_cv[
    [
        "params",
        "mean_test_score",
        "std_test_score",
        "rank_test_score"
    ]
].sort_values("rank_test_score")

df_lr_cv

In [ ]:
df_lr_cv["C"] = df_lr_cv["params"].apply(lambda x: x["clf__C"])
df_lr_cv["solver"] = df_lr_cv["params"].apply(lambda x: x["clf__solver"])

fig, axes = plt.subplots(1, 2, figsize=(12, 4), sharey=True)

solvers = ["lbfgs", "liblinear"]

for ax, solver in zip(axes, solvers):
    data_solver = df_lr_cv[df_lr_cv["solver"] == solver]

    sns.lineplot(
        data=data_solver,
        x="C",
        y="mean_test_score",
        marker="o",
        ax=ax
    )

    ax.set_xscale("log")
    ax.set_title(f"Solver = {solver}")
    ax.set_xlabel("C (Regularization strength)")
    ax.set_ylabel("Mean F1-macro (CV)")

    # ===== ANNOTATE ĐIỂM C = 1 =====
    best_point = data_solver[data_solver["C"] == 1].iloc[0]
    best_c = best_point["C"]
    best_score = best_point["mean_test_score"]

    ax.annotate(
        f"{best_score:.3f}",
        xy=(best_c, best_score),
        xytext=(best_c, best_score - 0.03),
        arrowprops=dict(arrowstyle="->"),
        ha="center",
        fontsize=9
    )

plt.suptitle("Effect of C on Logistic Regression (TF-IDF tuned)", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
#Thử trên tập test
lr_tuned_result = evaluate_model(
    grid_lr.best_estimator_,
    X_test,
    y_test
)
lr_tuned_result

In [ ]:
df_lr_compare = pd.DataFrame([
    {"Model": "Logistic Regression (baseline)", **lr_base_result},
    {"Model": "Logistic Regression (tuned)", **lr_tuned_result}
])

df_lr_compare

Phân tích ảnh hưởng các các từ (feature)

In [19]:
# Lấy vectorizer và model từ pipeline
tfidf = pipe_lr_base.named_steps["tfidf"]
clf = pipe_lr_base.named_steps["clf"]

feature_names = np.array(tfidf.get_feature_names_out())

coef = clf.coef_
classes = clf.classes_

In [20]:
top_n = 20

neg_idx = np.where(classes == "negative")[0][0]
top_neg = np.argsort(coef[neg_idx])[-top_n:]

pd.DataFrame({
    "feature": feature_names[top_neg],
    "weight": coef[neg_idx][top_neg]
}).sort_values(by="weight", ascending=False)

feature    weight
19         dở  3.856422
18       chán  2.906791
17         tệ  2.708163
16      không  2.568805
15       tiền  2.328948
14  thất_vọng  2.230524
13       nhạt  2.199034
12  không hay  2.054672
11        phí  1.868847
10        chả  1.787734
9          gì  1.752605
8    buồn_ngủ  1.717091
7         ngủ  1.673451
6        nhảm  1.647050
5     phim dở  1.634633
4         xàm  1.619928
3    phí tiền  1.555225
2         như  1.543452
1       dở_tệ  1.454521
0   nhạt_nhẽo  1.449913

In [24]:
mix_idx = np.where(classes == "mixed/neutral")[0][0]
top_mix = np.argsort(coef[mix_idx])[-top_n:]

pd.DataFrame({
    "feature": feature_names[top_mix],
    "weight": coef[mix_idx][top_mix]
}).sort_values(by="weight", ascending=False)

feature    weight
19          tạm  3.673813
18        nhưng  2.943967
17           ổn  2.678622
16  bình_thường  2.579845
15          hơi  2.573485
14         cũng  2.308260
13         chưa  2.000689
12     giải_trí  1.987077
11    không quá  1.971808
10         thôi  1.844304
9          được  1.708126
8           kết  1.607814
7            ok  1.606772
6      cũng tạm  1.542570
5     tuy_nhiên  1.358393
4     hay nhưng  1.344702
3        tạm ổn  1.330983
2           khá  1.255611
1           trừ  1.226707
0          bình  1.200600

In [22]:
pos_idx = np.where(classes == "positive")[0][0]
top_pos = np.argsort(coef[pos_idx])[-top_n:]

pd.DataFrame({
    "feature": feature_names[top_pos],
    "weight": coef[pos_idx][top_pos]
}).sort_values(by="weight", ascending=False)

feature    weight
19        hay  4.029266
18   phim hay  2.963498
17       cười  2.201209
16       đáng  2.159829
15        rất  2.018785
14       đỉnh  1.887605
13        hài  1.641344
12        đẹp  1.584009
11  tuyệt_vời  1.480602
10   đáng xem  1.474978
9        siêu  1.423586
8        khóc  1.279622
7         một  1.273113
6     10 điểm  1.260412
5       tuyệt  1.233712
4    hài_hước  1.230364
3   dễ_thương  1.205999
2         nữa  1.131955
1        cuốn  1.131257
0    xuất_sắc  1.070566

### SVM TUNING

In [ ]:
from sklearn.svm import LinearSVC

pipe_svm_base = Pipeline([
    ("tfidf", TfidfVectorizer(**best_tfidf_params)),
    ("clf", LinearSVC(
        class_weight="balanced"
    ))
])

pipe_svm_base.fit(X_train, y_train)

svm_base_result = evaluate_model(pipe_svm_base, X_test, y_test)
svm_base_result

In [ ]:
param_grid_svm = {
    "clf__C": [0.01, 0.1, 1, 10]
}

grid_svm = GridSearchCV(
    pipe_svm_base,
    param_grid_svm,
    scoring="f1_macro",
    cv=cv,
    n_jobs=-1
)

grid_svm.fit(X_train, y_train)
print(grid_svm.best_params_)
print(grid_svm.best_score_)

In [ ]:
df_svm_cv = pd.DataFrame(grid_svm.cv_results_)

df_svm_cv = df_svm_cv[
    [
        "params",
        "mean_test_score",
        "std_test_score",
        "rank_test_score"
    ]
].sort_values("rank_test_score")

df_svm_cv

In [ ]:
df_svm_cv["C"] = df_svm_cv["params"].apply(lambda x: x["clf__C"])
plt.figure(figsize=(6, 4))

sns.lineplot(
    data=df_svm_cv,
    x="C",
    y="mean_test_score",
    marker="o"
)

best_point = df_svm_cv.loc[df_svm_cv["mean_test_score"].idxmax()]
best_c = best_point["C"]
best_score = best_point["mean_test_score"]

plt.annotate(
    f"{best_score:.3f}",
    xy=(best_c, best_score),
    xytext=(best_c, best_score - 0.005),
    arrowprops=dict(arrowstyle="->"),
    ha="center",
    fontsize=9
)

plt.xscale("log")
plt.xlabel("C (Regularization strength)")
plt.ylabel("Mean F1-macro (CV)")
plt.title("Effect of C on Linear SVM (TF-IDF tuned)")

In [ ]:
#Thử trên tập test
svm_tuned_result = evaluate_model(
    grid_svm.best_estimator_,
    X_test,
    y_test
)

In [ ]:
df_svm_compare = pd.DataFrame([
    {"Model": "Linear SVM (baseline)", **svm_base_result},
    {"Model": "Linear SVM (tuned)", **svm_tuned_result}
])

df_svm_compare

### Naive Bayes Tuning

In [ ]:
pipe_nb_base = Pipeline([
    ("tfidf", TfidfVectorizer(**best_tfidf_params)),
    ("clf", MultinomialNB())
])

pipe_nb_base.fit(X_train, y_train)

nb_base_result = evaluate_model(pipe_nb_base, X_test, y_test)

In [ ]:
param_grid_nb = {
    "clf__alpha": [0.1, 0.5, 1.0, 2.0]
}

grid_nb = GridSearchCV(
    pipe_nb_base,
    param_grid_nb,
    scoring="f1_macro",
    cv=cv,
    n_jobs=-1
)

grid_nb.fit(X_train, y_train)
print(grid_nb.best_params_)
print(grid_nb.best_score_)

In [ ]:
df_nb_cv = pd.DataFrame(grid_nb.cv_results_)
df_nb_cv["alpha"] = df_nb_cv["params"].apply(lambda x: x["clf__alpha"])

df_nb_table = df_nb_cv[
    ["alpha", "mean_test_score", "std_test_score", "rank_test_score"]
].sort_values("mean_test_score", ascending=False)

df_nb_table

In [ ]:
plt.figure(figsize=(6, 4))

sns.lineplot(
    data=df_nb_cv,
    x="alpha",
    y="mean_test_score",
    marker="o"
)
best_point = df_nb_cv.loc[df_nb_cv["mean_test_score"].idxmax()]
best_alpha = best_point["alpha"]
best_score = best_point["mean_test_score"]

plt.annotate(
    f"{best_score:.3f}",
    xy=(best_alpha, best_score),
    xytext=(best_alpha, best_score - 0.01),
    arrowprops=dict(arrowstyle="->"),
    ha="center",
    fontsize=9
)

plt.xlabel("alpha (Laplace smoothing)")
plt.ylabel("Mean F1-macro (CV)")
plt.title("Effect of alpha on Naive Bayes (TF-IDF tuned)")
plt.tight_layout()
plt.show()

In [ ]:
nb_tuned_result = evaluate_model(
    grid_nb.best_estimator_,
    X_test,
    y_test
)

In [ ]:
df_nb_compare = pd.DataFrame([
    {"Model": "Naive Bayes (baseline)", **nb_base_result},
    {"Model": "Naive Bayes (tuned)", **nb_tuned_result}
])

df_nb_compare

In [ ]:
df_final = pd.concat([
    df_lr_compare[df_lr_compare["Model"].str.contains("tuned")],
    df_svm_compare[df_svm_compare["Model"].str.contains("tuned")],
    df_nb_compare[df_nb_compare["Model"].str.contains("tuned")]
], ignore_index=True)

df_final.sort_values("F1_macro", ascending=False)

In [ ]:
df_final_compare = pd.DataFrame([
    {"Model": "Logistic Regression", "Setting": "Baseline", **lr_base_result},
    {"Model": "Logistic Regression", "Setting": "Tuned",    **lr_tuned_result},

    {"Model": "Linear SVM", "Setting": "Baseline", **svm_base_result},
    {"Model": "Linear SVM", "Setting": "Tuned",    **svm_tuned_result},

    {"Model": "Naive Bayes", "Setting": "Baseline", **nb_base_result},
    {"Model": "Naive Bayes", "Setting": "Tuned",    **nb_tuned_result},
])

df_final_compare.sort_values("F1_macro", ascending=False).reset_index(drop=True)

In [ ]:
from sklearn.metrics import classification_report

best_svm = pipe_svm_base   # vì baseline tốt hơn tuned

y_pred_svm = best_svm.predict(X_test)

print("=== Linear SVM (Baseline – Best Model) ===")
print(classification_report(y_test, y_pred_svm))

In [ ]:
from sklearn.metrics import confusion_matrix

labels = ["negative", "mixed/neutral", "positive"]

cm = confusion_matrix(
    y_test,
    y_pred_svm,
    labels=labels
)


In [ ]:
cm_norm = cm.astype("float") / cm.sum(axis=1, keepdims=True)

plt.figure(figsize=(6, 5))

sns.heatmap(
    cm_norm,
    annot=True,
    fmt=".2f",
    cmap="Blues",
    xticklabels=labels,
    yticklabels=labels
)

plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("Normalized Confusion Matrix – Linear SVM (Baseline)")
plt.tight_layout()
plt.show()

###Old version

In [ ]:
from sklearn.metrics import classification_report

print("Logistic Regression")
print(classification_report(y_test, y_pred_lr))

print("SVM")
print(classification_report(y_test, y_pred_svm))

print("Naive Bayes")
print(classification_report(y_test, y_pred_nb))

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

models = {
    "Logistic Regression": y_pred_lr,
    "SVM": y_pred_svm,
    "Naive Bayes": y_pred_nb
}

results = []

for name, pred in models.items():
    results.append({
        "Model": name,
        "Accuracy": accuracy_score(y_test, pred),
        "F1_macro": f1_score(y_test, pred, average="macro")
    })

df_results = pd.DataFrame(results)
df_results

In [ ]:
train_idx = X_train.index
test_idx  = X_test.index

# feature phụ
X_train_num = dfML.loc[train_idx, ["length", "has_emoji"]]
X_test_num  = dfML.loc[test_idx,  ["length", "has_emoji"]]

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_num_scaled = scaler.fit_transform(X_train_num)
X_test_num_scaled  = scaler.transform(X_test_num)

In [ ]:
from scipy.sparse import hstack
#ghép thêm 2 feature vào

X_train_final = hstack([X_train_tfidf, X_train_num_scaled])
X_test_final  = hstack([X_test_tfidf,  X_test_num_scaled])

In [ ]:
model = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",
    n_jobs=-1
)

model.fit(X_train_final, y_train)

y_pred_lr2 = model.predict(X_test_final)

print("===== TF-IDF only =====")
print(classification_report(y_test, y_pred_lr))

print("===== Khi thêm feature =====")
print(classification_report(y_test, y_pred_lr2))


In [ ]:
f1_text_only = f1_score(y_test, y_pred_lr, average="macro")
f1_text_plus = f1_score(y_test, y_pred_lr2, average="macro")

df_cmp = pd.DataFrame({
    "Model": ["TF-IDF", "TF-IDF + length + emoji"],
    "F1_macro": [f1_text_only, f1_text_plus]
})

plt.figure(figsize=(6,4))
sns.barplot(data=df_cmp, x="Model", y="F1_macro")
plt.ylim(0,1)
plt.title("So sánh hiệu năng mô hình")
plt.tight_layout()
plt.show()

In [ ]:
feature_names = (
    tfidf.get_feature_names_out().tolist()
    + ["length", "has_emoji"]
)

coef = model.coef_

df_coef = pd.DataFrame(
    coef.T,
    index=feature_names,
    columns=model.classes_
)

df_coef.loc[["length", "has_emoji"]]